In [1]:
import pandas as pd
from xgboost import XGBRegressor
import joblib
import category_encoders as ce
from sklearn.model_selection import train_test_split
import os
from pathlib import Path
from datetime import datetime
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error,r2_score,root_mean_squared_error
import h2o
from h2o.automl import H2OAutoML
import json,shutil
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
# Load environment variables
load_dotenv()
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST")
port = os.getenv("MYSQL_PORT")
db = os.getenv("MYSQL_DB")


# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}")

In [3]:
# Query recent records from car_data table
query = "SELECT * FROM car_data WHERE updated_at >= NOW() - INTERVAL 7 DAY"
df = pd.read_sql(query, engine)

In [4]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand,updated_at
0,4 Series,2016,15500.0,Semi-Auto,59461,Diesel,30.0,65.7,2.0,BMW,2025-05-31 13:10:08
1,Fiesta,2014,7500.0,Semi-Auto,25211,Petrol,0.0,57.7,1.0,Ford,2025-05-31 13:10:08
2,Grand C-MAX,2017,9690.0,Manual,40911,Diesel,150.0,64.2,1.5,Ford,2025-05-31 13:10:08
3,Aygo,2017,7400.0,Manual,5985,Petrol,0.0,69.0,1.0,toyota,2025-05-31 13:10:08
4,Polo,2018,16995.0,Semi-Auto,16940,Petrol,145.0,47.9,2.0,VW,2025-05-31 13:10:08
